## Dependencies

In [1]:
# Confirmed needed dependencies
import pprint
import pandas as pd
import requests
import json

# Dependencies for geocoordinates generator
import sys
import random
import math
import gmplot

# Dependencies for conversion of coordinates to addresses
import geopy
from geopy.geocoders import Nominatim
import time

# Dependencies for Zillow data
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults

# Dependency for Heat Mapper
import gmaps


# Add config.py file with the following variables and cooresponding Zillow API keys
from config import Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta
from config import google_API_Key


################# ONGOING EDITS TO REQUIREMENTS.MD #################
###### IF ANY ERRORS OCCUR RELATING TO MODULES OR CONFIG.PY #######
### REFER TO requirements.md TO ENSURE YOU ARE PROPERLY SETUP ####

## File inputs/outputs

In [2]:
randLatLon_csv = "./Data/randomLatLon.csv" 
addressList_csv = "./Data/addressList.csv"
masterData_csv = "./Data/masterData.csv"

## Geocoordinates of Austin

In [ ]:
##########################################
# this section written by troy bailey.   #
# enter uservariables below to determine #
# center location, radius of circle, and #
# number of geocoordinates to generate.  #
##########################################

In [ ]:
########################
#### USER VARIABLES ####
########################

x0 = 30.27444       #### Set center coordiantes in decimal degrees
y0 = -97.74028      #### initial coordiantes are location of Texas State Capitol Building

radius = 20         #### Set radius in miles

points = 40000        #### Set number of lat,lon points to generate

In [ ]:
# variables and inputs for coordinate calculations
lat_lon_list = []
radiusInDegrees=radius/69           
r = radiusInDegrees
points += 1

In [ ]:
# calculate each coordiante point and build a list of lat and lon
for i in range(1, points):
    u = float(random.uniform(0.0,1.0)) #random number for radius length
    v = float(random.uniform(0.0,1.0)) #random number for pi radians
    
    w = r * math.sqrt(u) #radius length
    t = 2 * math.pi * v  #radians
    x = w * math.cos(t)  #calculate x coord distance
    y = w * math.sin(t)  #calculate y coord distance
    
    xLat  = x + x0       #offset x by center x
    yLon = y + y0        #offset y by center y
    
    lat_lon_list.append([xLat,yLon])

# convert list to dataframe
lat_lon_df = pd.DataFrame(lat_lon_list, columns=['lat','lon'])

lat_lon_df.head()

len(lat_lon_df)

In [ ]:
# write a CSV file of coordinate points
lat_lon_df.to_csv(randLatLon_csv, index=False, header=True)

## Plot coordinate points on map


In [ ]:
# This section will plot points on a Google map centered at centerPointLat and centerPointLon with a magnification of magFactor
# It assumes there is a dataframe with "lat" and "lon" columns
# The resulting map is saved to a file called 

centerPointLat = 30.27444  #these are the coordinates of the Texas State Capitol building
centerPointLon = -97.74028 #these are the coordinates of the Texas State Capitol building
magnificationFactor = 10
pointColor = "red"
pointSize = 100
mapOutputFile = "mymap.html"
df = lat_lon_df

gmap = gmplot.GoogleMapPlotter(centerPointLat, centerPointLon, magnificationFactor)

gmap.scatter(df["lat"], df["lon"], pointColor, size=pointSize, marker=False)

gmap.draw("./Visuals/" + mapOutputFile)

## Convert Coordinates to Residential Addresses

In [ ]:
#########################
##### Yuta's Blocks #####
#########################

##### Geopy Nominatim API #####
geopy.geocoders.options.default_user_agent = "ut-group-EPIC"

pp = pprint.PrettyPrinter(indent=4)

url = "https://nominatim.openstreetmap.org/reverse?"

In [ ]:
# Test API - Known Residential Address
params_1 = {
    "format": "jsonv2",
    "lat": 30.440777,
    "lon": -97.777048
}

print("===== Test Home Response:")
response = requests.get(url, params=params_1).json()
pp.pprint(response)
print("\n" + "="*60 + "\n")

In [ ]:
# Import CSV, put into DataFrame
latlon_df = pd.read_csv(randLatLon_csv)
latlon_df.head()

In [ ]:
# Put latitudes and longitudes into a zip object
lats = latlon_df.iloc[:, 0]
lons = latlon_df.iloc[:, 1]
lat_lons = []
lat_lons = zip(lats, lons)

In [ ]:
##### Loop Request API for Addresses / Append to lists #####
# Make sure to import time

query_url = "https://nominatim.openstreetmap.org/reverse?"

house_num = []
road = []
postcode = []
lat = []
lon = []
neighborhood = []

counter = 1
numRequests = latlon_df["lat"].count()
rSuccess = []
rFailure = []

print(f"Processing {numRequests} Requests...")

# Nominatim API Request

for lat_lon in lat_lons:
    params = {
        "format": "jsonv2",
        "lat": lat_lon[0],
        "lon": lat_lon[1]
    }

    time.sleep(1.1)
    response = requests.get(query_url, params=params).json()

    if response['type'] == 'house' or response['type'] == 'yes':
        lat.append(response['lat'])
        lon.append(response['lon'])
        
        try:
            postcode.append(response['address']['postcode'])
        except (KeyError, IndexError):
            postcode.append("NA")
        try:
            house_num.append(response['address']['house_number'])
        except (KeyError, IndexError):
            house_num.append("NA")
        try:
            road.append(response['address']['road'])
        except (KeyError, IndexError):
            road.append("NA")
        try:
            neighborhood.append(response['address']['neighbourhood'])
        except (KeyError, IndexError):
            neighborhood.append("NA")
        
        print(f"Processed Record {counter} of {numRequests}.")
        rSuccess.append(counter)
        counter += 1
        
    else:
        print(f"Wrong Type - Skipped Record {counter} of {numRequests}.")
        rFailure.append(counter)
        counter += 1
        
print(f"Finished Requests !!!")

In [ ]:
print("Request Results:")
print("Success #:" + str(len(rSuccess)))
print("Skipped #:" + str(len(rFailure)))

In [ ]:
# Create dataframe with addresses from API requests
address_df = pd.DataFrame({
    "house #": house_num,
    "street": road,
    "zipcode": postcode,
    "lat": lat,
    "lon": lon,
    "neighborhood": neighborhood,
})

# Clean up Dataframe Columns before output (Drop incomplete zipcodes, Highway streets, and Null house # or streets)
address_df = address_df[address_df['zipcode'].str.len() == 5]
address_df = address_df[address_df['zipcode'].apply(lambda x: len(str(x)) > 3)]
address_df = address_df[address_df['street'].str.contains("Highway") == False]
address_df = address_df[address_df['house #'].str.contains("NA") == False]
address_df = address_df[address_df['street'].str.contains("NA") == False]
address_df.dtypes

In [ ]:
# write a CSV file of addresses
address_df.to_csv(addressList_csv, index=False, header=True)

In [ ]:
# Map out CSV with gmplot

addressList_csv = "./Data/addressList.csv"

gmap = gmplot.GoogleMapPlotter(30.27444, -97.74028, 10)

gmap.scatter(addressList_csv["lat"], addressList_csv["lon"], 'red', size=20, marker=False)

gmap.draw("./Visuals/myaddressmap.html")

## Zillow API Calls using Address and Zipcode

In [3]:
#############################
##### VALERIE'S BLOCKS #####
###########################

# Function for reading in csv, checking for headers, and appending if appropriate
def csvDFappend(oldCSVfilepath, newColumn):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    # Checking to ensure new header name does not match any current headers
    colNames = DF.columns
    for value in colNames:
        if value == newColumn:
            print("Cannot append column that matches an existing column name")
            return DF
    # Check to ensure length of newColumn matches length of current dataframe columns
    if len(newColumn) != len(DF):
        print("Cannot append column that is not the same length as existing dataframe")
        return DF
    # Append newColumn to Dataframe
    DF[newColumn] = newColumn
    return DF

In [4]:
### TEST BLCOK FOR FUNCTION ELEMENTS OF csvDFappend ###

colNames = masterDFIMPORTclean.columns
newCol = "zipcode"
for value in colNames:
    print(value)
    if value == newCol:
        print("Cannot append column that matches an existing column name")
        break
        
if len(address_df["zipcode"]) != len(masterDFIMPORTclean):
    print("Cannot append column that is not the same length as existing dataframe")  

NameError: name 'masterDFIMPORTclean' is not defined

In [ ]:
# Tiny sample to work with looping without exhausting API call limits
addressListTiny_csv = "./Data/addressListTiny.csv"

address_sample = pd.read_csv(addressListTiny_csv)
address_df = pd.DataFrame(address_sample)
print(len(address_df))

In [4]:
# Funtion for reading CSV in as DataFrame
def csvDF(oldCSVfilepath):
    csvIN = pd.read_csv(oldCSVfilepath)
    DF = pd.DataFrame(csvIN)
    return DF

# Function for converting DataFrame to CSV
### IN PROCESS ###
def DFcsv(dataframe, newCSVfilepath):
    newCSVfilepath = pd.to_csv(DF)
    return DF

In [6]:
addressDF = csvDF(addressList_csv)
print(len(addressDF))

20177


In [7]:
############### LOOPING FUNCTION FULLY OPERATIONAL ################
####### HOWEVER, ZILLOW ONLY ALLOWS 1000 API CALLS PER DAY #######

# Zillow API call function
def zCall(API, index, address, zipcode):
    APIkey = API[index]
    zillow_data = ZillowWrapper(APIkey)
    deep_search_response = zillow_data.get_deep_search_results(address, zipcode)
    result = GetDeepSearchResults(deep_search_response)
    return result

# List containers for collected property data
zid = []
addresses = []
alats = []
alons = []
valuation = []
sqft = []

# List of Zillow API keys to loop through due to daily API call limits
zAPIs = [Zapi, Zapi01, Zapi02, Zapi03, Zapi04, Zapi05, Zapi06, Zapi07, Zapi08, Zapi09, 
         Zapi10, Zapi11, Zapi12, Zapi13, Zapi14, Zapi15, Zapi16, Zapi17, Zapi18, Zapi19, 
         Zapi20, Ztroy1, Ztroy2, Ztroy3, Zseth, Zseth2, Zkat, Zval, Zyuta]
index = 0
    
for row, home in addressDF.iterrows():
    address = str(addressDF["house #"][row]) + " " + str(addressDF["street"][row])
    addresses.append(address)
    zipcode = addressDF["zipcode"][row]
    print(f"Processing {address}, {zipcode} (index {row}).")
    
    result = None
    try:
        try:
            result = zCall(zAPIs, index, address, zipcode)
            print(f"{row} Success!")
        except KeyError:  ### ERROR FOR API CALL LIMIT EXCEEDED ###
            print(f"KeyError has occurred for {address}, {zipcode} (index {row}).")
            index += 1
            print(f"Proceeding to API[{index}]")
            if index >= len(zAPIs):
                print(f"API[{index}] does not exist. Need more API keys to complete analysis.")
                break
            result = zCall(zAPIs, index, address, zipcode)

    except:
        print(f"No record found for {address}, {zipcode} (index {row}). Appending lists with null values")
        zid.append(None)
        alats.append(None)
        alons.append(None)
        valuation.append(None)
        sqft.append(None)
        continue

    try:
        zillowID = result.zillow_id
        zid.append(zillowID)
    except:
        print(f"No zid found for {address}, {zipcode} (index {row}). Appending list with null values")
        zid.append(None)

    try:
        alat = result.latitude
        alats.append(alat)
    except:
        print(f"No alat found for {address}, {zipcode} (index {row}). Appending list with null values")
        alats.append(None)

    try:
        alon = result.longitude
        alons.append(alon)
    except:
        print(f"No alon found for {address}, {zipcode} (index {row}). Appending list with null values")
        alons.append(None)

    try:    
        val = int(result.zestimate_amount)
        valuation.append(val)
    except:
        print(f"No valuation found for {address}, {zipcode} (index {row}). Appending list with null values")
        valuation.append(None)

    try:
        zsqft = int(result.home_size)
        sqft.append(zsqft)
    except:
        print(f"No sqft found for {address}, {zipcode} (index {row}). Appending list with null values")
        sqft.append(None)

Processing 11508 Carl Road, 78747 (index 0).
KeyError has occurred for 11508 Carl Road, 78747 (index 0).
Proceeding to API[1]
No record found for 11508 Carl Road, 78747 (index 0). Appending lists with null values
Processing 11206 Lime Creek Road, 78641 (index 1).
KeyError has occurred for 11206 Lime Creek Road, 78641 (index 1).
Proceeding to API[2]
No record found for 11206 Lime Creek Road, 78641 (index 1). Appending lists with null values
Processing 2905 Etta James Cove, 78613 (index 2).
KeyError has occurred for 2905 Etta James Cove, 78613 (index 2).
Proceeding to API[3]
No record found for 2905 Etta James Cove, 78613 (index 2). Appending lists with null values
Processing 607 River Road, 78734 (index 3).
KeyError has occurred for 607 River Road, 78734 (index 3).
Proceeding to API[4]
No record found for 607 River Road, 78734 (index 3). Appending lists with null values
Processing 15029 Donna Jane Loop, 78660 (index 4).
KeyError has occurred for 15029 Donna Jane Loop, 78660 (index 4).
P

No record found for 1658 Courtney Lane, 78613 (index 63). Appending lists with null values
Processing 12400 Mellow Meadows, 78729 (index 64).
No record found for 12400 Mellow Meadows, 78729 (index 64). Appending lists with null values
Processing 12701 Johnson Road, 78653 (index 65).
65 Success!
Processing 909 Terrace Mountain Drive, 78746 (index 66).
66 Success!
Processing 6348 Elroy Road, 78617 (index 67).
No record found for 6348 Elroy Road, 78617 (index 67). Appending lists with null values
Processing 2200 Birdell Street, 78734 (index 68).
No record found for 2200 Birdell Street, 78734 (index 68). Appending lists with null values
Processing 22300 Ann Showers Drive, 78621 (index 69).
69 Success!
Processing 512 Congress Avenue, 78701 (index 70).
70 Success!
No valuation found for 512 Congress Avenue, 78701 (index 70). Appending list with null values
Processing 4501 RM 620, 78732 (index 71).
No record found for 4501 RM 620, 78732 (index 71). Appending lists with null values
Processing 

No record found for 12008 US 290, 78737 (index 139). Appending lists with null values
Processing 3811 Tarragona Lane, 78727 (index 140).
140 Success!
Processing 1601 Acacia Bud Drive, 78733 (index 141).
141 Success!
Processing 9708 Schmidt Lane, 78653 (index 142).
142 Success!
Processing 1580 Medical Parkway, 78613 (index 143).
No record found for 1580 Medical Parkway, 78613 (index 143). Appending lists with null values
Processing 11217 Crumley Ranch Road, 78736 (index 144).
No record found for 11217 Crumley Ranch Road, 78736 (index 144). Appending lists with null values
Processing 5611 Colton Road, 78719 (index 145).
No record found for 5611 Colton Road, 78719 (index 145). Appending lists with null values
Processing 4412 Corran Ferry Loop, 78749 (index 146).
146 Success!
Processing 9916 Horizon Lane, 78719 (index 147).
147 Success!
Processing 16901 Gravesend Road, 78660 (index 148).
148 Success!
Processing 3510 Pawnee Pass, 78738 (index 149).
149 Success!
Processing 2943 Thurman Road,

No record found for 1594 Wood Acre Lane, 78733 (index 225). Appending lists with null values
Processing 11607 Buttonwood Drive, 78759 (index 226).
226 Success!
Processing 4210 Turnersville Road, 78610 (index 227).
227 Success!
Processing 2713 La Mesa Drive, 78704 (index 228).
228 Success!
Processing 11104 FM 1625, 78719 (index 229).
229 Success!
Processing 131 Holz Road, 78640 (index 230).
No record found for 131 Holz Road, 78640 (index 230). Appending lists with null values
Processing 14609 Pocohontas Trail, 78641 (index 231).
231 Success!
Processing 19600 Moorlynch Avenue, 78660 (index 232).
232 Success!
Processing 269 West Old Bowman Road, 78664 (index 233).
No record found for 269 West Old Bowman Road, 78664 (index 233). Appending lists with null values
Processing 1088 Breakaway Road, 78613 (index 234).
No record found for 1088 Breakaway Road, 78613 (index 234). Appending lists with null values
Processing 10103 Lindshire Lane, 78748 (index 235).
235 Success!
Processing 641 Hurst Cr

303 Success!
Processing 12517 Harmony Hill Street, 78729 (index 304).
No record found for 12517 Harmony Hill Street, 78729 (index 304). Appending lists with null values
Processing 13115 FM 812, 78719 (index 305).
No record found for 13115 FM 812, 78719 (index 305). Appending lists with null values
Processing 9105 Chisholm Lane, 78748 (index 306).
306 Success!
Processing 10304 Wolftrap Drive, 78749 (index 307).
307 Success!
Processing 12445 Fairfax Ridge Place, 78738 (index 308).
308 Success!
Processing 100 Misty Lane, 78612 (index 309).
No record found for 100 Misty Lane, 78612 (index 309). Appending lists with null values
Processing 14099 Ashton Woods Circle, 78727 (index 310).
310 Success!
Processing 170 Story Drive, 78610 (index 311).
No record found for 170 Story Drive, 78610 (index 311). Appending lists with null values
Processing 12601 Maidenhair Lane, 78738 (index 312).
312 Success!
Processing 4807 Misty Brook Cove, 78727 (index 313).
313 Success!
Processing 17300 Littig Road, 7

...

No record found for 1522 Weatherford Drive, 78667 (index 19824). Appending lists with null values
Processing 12016 Trails End Road, 78641 (index 19825).
19825 Success!
Processing 1102 Brentwood Street, 78757 (index 19826).
19826 Success!
Processing 4611 Fallenash Drive, 78725 (index 19827).
19827 Success!
Processing 15099 Warbler Drive, 78734 (index 19828).
19828 Success!
No valuation found for 15099 Warbler Drive, 78734 (index 19828). Appending list with null values
No sqft found for 15099 Warbler Drive, 78734 (index 19828). Appending list with null values
Processing 222 Neville Wood Court, 78738 (index 19829).
No record found for 222 Neville Wood Court, 78738 (index 19829). Appending lists with null values
Processing 11608 Hollister Drive, 78737 (index 19830).
19830 Success!
Processing 7103 Meadowood Drive, 78723 (index 19831).
19831 Success!
Processing 8413 Asmara Drive, 78750 (index 19832).
19832 Success!
Processing 8525 Clinger Road, 78719 (index 19833).
No record found for 8525 C

19900 Success!
No valuation found for 1011 Springdale Road, 78721 (index 19900). Appending list with null values
Processing 22551 Hog Eye Road, 78653 (index 19901).
19901 Success!
No valuation found for 22551 Hog Eye Road, 78653 (index 19901). Appending list with null values
No sqft found for 22551 Hog Eye Road, 78653 (index 19901). Appending list with null values
Processing 15981 Cinca Terra Drive, 78738 (index 19902).
No record found for 15981 Cinca Terra Drive, 78738 (index 19902). Appending lists with null values
Processing 3906 Houston Road, 78617 (index 19903).
19903 Success!
Processing 4605 Englewood Drive, 78745 (index 19904).
19904 Success!
Processing 2541 Arbor Drive, 78681 (index 19905).
19905 Success!
Processing 702 Knollwood Circle, 78746 (index 19906).
19906 Success!
Processing 2001 Winsted Lane, 78703 (index 19907).
19907 Success!
Processing 2253 North Cannes Drive, 78613 (index 19908).
No record found for 2253 North Cannes Drive, 78613 (index 19908). Appending lists wit

No record found for 10010 Ranch to Market Road 2222, 78750 (index 19974). Appending lists with null values
Processing 14109 Maricella Lane, 78660 (index 19975).
No record found for 14109 Maricella Lane, 78660 (index 19975). Appending lists with null values
Processing 1701 Travis Heights Boulevard, 78704 (index 19976).
19976 Success!
Processing 15408 Fernhill Drive, 78717 (index 19977).
19977 Success!
Processing 1000 Alta Vista Avenue, 78704 (index 19978).
19978 Success!
Processing 9404 Ledgestone Terrace, 78737 (index 19979).
19979 Success!
Processing 8300 Long Canyon Drive, 78730 (index 19980).
19980 Success!
Processing 3901 Norwood Lane, 78617 (index 19981).
No record found for 3901 Norwood Lane, 78617 (index 19981). Appending lists with null values
Processing 17131 Rocky Ridge Road, 78734 (index 19982).
19982 Success!
Processing 9101 Spinning Leaf Cove, 78735 (index 19983).
19983 Success!
Processing 9907 Lorado Drive, 78725 (index 19984).
No record found for 9907 Lorado Drive, 78725

20056 Success!
Processing 15230 Farm-to-Market Road 973, 78653 (index 20057).
No record found for 15230 Farm-to-Market Road 973, 78653 (index 20057). Appending lists with null values
Processing 5304 Honey Dew Terrace, 78749 (index 20058).
20058 Success!
Processing 349 Escondido Drive, 78617 (index 20059).
No record found for 349 Escondido Drive, 78617 (index 20059). Appending lists with null values
Processing 1804 Rabb Road, 78704 (index 20060).
20060 Success!
Processing 7416 Elroy Road, 78617 (index 20061).
20061 Success!
No valuation found for 7416 Elroy Road, 78617 (index 20061). Appending list with null values
Processing 217 Bella Riva Drive, 78734 (index 20062).
20062 Success!
Processing 12521 Amherst Drive, 78727 (index 20063).
No record found for 12521 Amherst Drive, 78727 (index 20063). Appending lists with null values
Processing 6815 Side Saddle Street, 78745 (index 20064).
20064 Success!
Processing 2201 Nan Lane, 78613 (index 20065).
No record found for 2201 Nan Lane, 78613 (

20134 Success!
No valuation found for 8507 Kromer Street, 78757 (index 20134). Appending list with null values
Processing 2116 Maple Vista Drive, 78660 (index 20135).
20135 Success!
Processing 4340 RM 620, 78732 (index 20136).
No record found for 4340 RM 620, 78732 (index 20136). Appending lists with null values
Processing 1748 Wild Horse Lane, 78681 (index 20137).
No record found for 1748 Wild Horse Lane, 78681 (index 20137). Appending lists with null values
Processing 10515 River Plantation Drive, 78747 (index 20138).
20138 Success!
Processing 7306 Bennett Avenue, 78752 (index 20139).
20139 Success!
Processing 667 Barton Creek Boulevard, 78746 (index 20140).
No record found for 667 Barton Creek Boulevard, 78746 (index 20140). Appending lists with null values
Processing 6803 Bill Hughes Road, 78745 (index 20141).
20141 Success!
Processing 5406 North Lamar Boulevard, 78756 (index 20142).
No record found for 5406 North Lamar Boulevard, 78756 (index 20142). Appending lists with null valu

In [8]:
print(len(addresses))

20177


## Master Dataframe Creation

In [9]:
# Checking to ensure lists are appropriate lengths
print(len(zid))
print(len(alats))
print(len(alons))
print(len(addresses))
print(len(valuation))
print(len(sqft))
print(len(valsqft))

# Referring back to addressList_csv generated dataframe for relevant info
addressDF.head()

20177
20177
20177
20177
20177
20177


NameError: name 'valsqft' is not defined

In [10]:
# FULL SAMPLE FROM 40,000 RANDOM LAT, LON GENERATION

masterDF = pd.DataFrame({
    "Zillow ID": zid,
    "address": addresses,
    "zipcode": addressDF["zipcode"],
    "alat": alats,
    "alon": alons,
    "valuation": valuation,
    "sqft": sqft,
#     "value sqft": valsqft,
    "neighborhood": addressDF["neighborhood"],
})
print(len(masterDF))
masterDF.head(10)

20177


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
0,None,11508 Carl Road,78747,None,None,NaN,NaN,NaN
1,None,11206 Lime Creek Road,78641,None,None,NaN,NaN,NaN
2,None,2905 Etta James Cove,78613,None,None,NaN,NaN,NaN
3,None,607 River Road,78734,None,None,NaN,NaN,NaN
4,None,15029 Donna Jane Loop,78660,None,None,NaN,NaN,Spring Hill Village
5,None,12700 Johnson Lane,78652,None,None,NaN,NaN,Olympic Heights
6,None,13805 Abrahamson Court,78653,None,None,NaN,NaN,NaN
7,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
8,None,4105 Aqua Verde Drive,78746,None,None,NaN,NaN,NaN
9,None,10555 Maha Circle,78747,None,None,NaN,NaN,NaN


In [11]:
masterDFclean = masterDF.dropna(how="any", subset=["Zillow ID"])
len(masterDFclean)

11805

In [13]:
# masterDF to csv
masterDFclean.to_csv(masterData_csv, index=False, header=True)
masterDFclean.head(30)

,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
7,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
12,29440641,2109 Red Stone Lane,78727,30.418454,-97.69619,249696.0,1359.0,Tomanet Estates
14,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,NaN
15,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,Hunter's Chase
16,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,North Cat Mountain
17,29471589,11125 Maha Loop Road,78617,30.082419,-97.664366,212919.0,1960.0,NaN
18,29352791,102 Javelin Drive,78734,30.360267,-97.998646,370772.0,2088.0,NaN
21,64884422,7220 Cielo Azul Pass,78726,30.41662,-97.874616,2592410.0,6060.0,NaN
22,29417097,2000 Brentwood Street,78757,30.340507,-97.734626,445342.0,1051.0,Abercrombie
23,83820494,12220 Dwight Eisenhower Street,78653,30.354256,-97.526464,NaN,1512.0,Presidential Meadows


In [ ]:
### CONTINUE HERE READ IN NEW CSV TO CLEAN
### pasted from top of file for reference ### masterData_csv = "./Data/masterData.csv"

In [9]:
masterDF = csvDF(masterData_csv)
print(len(masterDF))
masterDF.head()

11805


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,Tomanet Estates
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,NaN
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,Hunter's Chase
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,North Cat Mountain


In [19]:
# DATA CLEANING #
# Dropping duplicates
masterDFdrops = masterDF.drop_duplicates(subset=["Zillow ID"], keep="first")
print(len(masterDFdrops))
masterDFdrops.head()

9697


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,Tomanet Estates
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,NaN
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,Hunter's Chase
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,North Cat Mountain


In [20]:
# Cleaning out None values for "valuation"
masterDFdrops = masterDFdrops.dropna(how="any", subset=["valuation"])
print(len(masterDFdrops))
masterDFdrops.head()

8752


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,Tomanet Estates
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,NaN
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,Hunter's Chase
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,North Cat Mountain


In [21]:
# Cleaning out None values for "sqft" 
masterDFdrops = masterDFdrops.dropna(how="any", subset=["sqft"])
print(len(masterDFdrops))
masterDFdrops.head()

8667


,Zillow ID,address,zipcode,alat,alon,valuation,sqft,neighborhood
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,Cherry Creek
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,Tomanet Estates
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,NaN
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,Hunter's Chase
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,North Cat Mountain


## Calculate Value per Sqft

In [22]:
# Calculate "value sqft" after None value rows removed for "valuation" and "sqft"

valsqft = []
for row, value in masterDFdrops.iterrows():
    try:
        vsqft = round((masterDFdrops["valuation"][row] / masterDFdrops["sqft"][row]), 2)
        valsqft.append(vsqft)
    except: ### THIS ERROR SHOULD NO LONGER PRINT BECAUSE NONE VALUES WERE PREVIOUSLY REMOVED
        print("Cannot perform math with NoneType")
        valsqft.append(None)

In [23]:
# Checking to ensure lists are appropriate lengths
print(len(masterDFdrops))
print(len(valsqft))

8667
8667


In [24]:
# Adding "value sqft" column
masterDFdrops["value sqft"] = valsqft

# Reordering columns
masterDFdrops = masterDFdrops[['Zillow ID', 'address', 'zipcode', 'alat', 'alon', 'valuation', 'sqft', 'value sqft', 'neighborhood']]
masterDFdrops.head()

Index(['Zillow ID', 'address', 'zipcode', 'alat', 'alon', 'valuation', 'sqft',
       'neighborhood', 'value sqft'],
      dtype='object')

In [35]:
# Checking high and low values with sort
masterDFIMPORTclean.sort_values(by="value sqft", ascending=False)

,Zillow ID,address,zipcode,alat,alon,valuation,sqft,value sqft,neighborhood
8297,29429760,8615 Rockwood Lane,78757,30.368305,-97.729611,9182418.0,560.0,16397.18,North Shoal Creek
9058,29323724,516 Dawson Road,78704,30.259646,-97.756519,9658799.0,785.0,12304.20,Bouldin
3348,29332184,1315 Norwalk Lane,78703,30.289010,-97.771691,6951266.0,683.0,10177.55,Westfield
4127,144999861,8200 South Congress Avenue,78745,30.176832,-97.785777,4037449.0,858.0,4705.65,South Bend
2556,144948842,11961 Overlook Pass,78738,30.278458,-97.933508,4272066.0,987.0,4328.33,NaN
4181,64713674,719 Shady Lane,78702,30.254580,-97.695101,7175807.0,1878.0,3820.98,Govalle
9679,120900802,303 East Pflugerville Parkway,78660,30.462785,-97.626114,7335401.0,2242.0,3271.81,Swenson Farms
5216,124720318,18498 Drecker Springs Lane,78660,30.451651,-97.574161,6871485.0,2152.0,3193.07,Villages of Hidden Lake
2147,64846259,1404 Rockcliff Road,78746,30.335784,-97.785358,9166290.0,3027.0,3028.18,Cat Mountain
2233,144952872,4029 Pearce Road,78730,30.346609,-97.812309,3767018.0,1300.0,2897.71,NaN


In [30]:
# Function for finding and dropping rows with nonsense values (e.g. humidity > 100)
def dropNonsense(dataframe, columnName, minVal, maxVal):
    dropIndices = []
    for index, row in dataframe.iterrows():
        val = dataframe[columnName][index]
        if (maxVal is not None) and (val > maxVal) or (minVal is not None) and (val < minVal):
            dropIndices.append(index)
    return dataframe.drop(index=dropIndices)

In [31]:
# Establish reasonable values for columns
reasonableVals = [
    ("valuation", 20000, 10000000),
    ("sqft", 500, 10000)
#     ("value sqft", 0, 100)
]

# Loop through dataframe to drop nonsense data
masterDFIMPORTclean = masterDFdrops
for entry in reasonableVals:
    masterDFIMPORTclean = dropNonsense(masterDFIMPORTclean, entry[0], entry[1], entry[2])

# Confirm count
len(masterDFIMPORTclean)

8552

In [36]:
masterDFIMPORTclean.to_csv("./Data/masterDFIMPORTclean.csv", index=False, header=True)
masterDFIMPORTclean.head(10)

,Zillow ID,address,zipcode,alat,alon,valuation,sqft,value sqft,neighborhood
0,29498621,8113 West Gate Boulevard,78745,30.197589,-97.828106,265572.0,940.0,282.52,Cherry Creek
1,29440641,2109 Red Stone Lane,78727,30.418454,-97.696190,249696.0,1359.0,183.74,Tomanet Estates
2,29345444,9 Glen Rock Drive,78738,30.341187,-97.984049,472040.0,2896.0,163.00,NaN
3,29613796,8333 Alvin High Lane,78729,30.460441,-97.761118,307384.0,1983.0,155.01,Hunter's Chase
4,29358809,5300 Valburn Circle,78731,30.374096,-97.775259,941669.0,4376.0,215.19,North Cat Mountain
5,29471589,11125 Maha Loop Road,78617,30.082419,-97.664366,212919.0,1960.0,108.63,NaN
6,29352791,102 Javelin Drive,78734,30.360267,-97.998646,370772.0,2088.0,177.57,NaN
7,64884422,7220 Cielo Azul Pass,78726,30.416620,-97.874616,2592410.0,6060.0,427.79,NaN
8,29417097,2000 Brentwood Street,78757,30.340507,-97.734626,445342.0,1051.0,423.73,Abercrombie
10,144341633,1253 Dunlap Road South,78725,30.222870,-97.566622,361665.0,2550.0,141.83,Chaparral Crossing


## Crime Data

In [ ]:
# Kat's section

## School Data

In [ ]:
# Seth's section

## Heat Mapper

In [ ]:
# Troy's section


gmaps.configure(api_key=google_API_Key)

In [ ]:
# This cell creates a test masterData_df by pulling in Yuta's address file and adds a column as a testm "value to map"
# This cell can be deleted as soon as there is a master data file that includes a property value column or some other value to plot
# The last digit of the zipcode is used as a value that will vary by area and a random number between 0 and 1 is added to create variation in the weights

masterData_df = pd.read_csv(addressList_csv)
zips = masterData_df["zipcode"]
valueToMap = []

for zip in zips:
    lastDigit = zip[-1:]
#    print(last2Digits)
    valueToMap.append(int(lastDigit) + random.uniform(0.0,1.0))
    
masterData_df["valueToMap"] = valueToMap
masterData_df.head()

In [ ]:
# This cell uses gmaps library to create a google heat map from the data in a master data file.
# The masterData csv file is taken as input
# The lat and lon columns are taken as the coordinates for hte heatmap 
# The user specified column is taken as the weighting valies fo each coordinate point

df = masterData_df
columnToMap = 'valueToMap'
max_intensity = df[columnToMap].max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
fig.add_layer(heatmap_layer)
fig

In [ ]:
# this is a function version of the cell above
# the function takes columnToMap as the weights for the points defined by 'lat' and 'lon' columns in the dataframe
# the dataframe can be included as a parameter, if it is not included masterData_df is assumed

def heatMapper(columnToMap, df = masterData_df):
    
    max_intensity = df[columnToMap].max()
    
    fig = gmaps.figure()
    heatmap_layer = gmaps.heatmap_layer(df[['lat', 'lon']], weights=df[columnToMap], max_intensity=max_intensity, point_radius=10.0)
    fig.add_layer(heatmap_layer)

    return;

In [ ]:
heatMapper(columnToMap = 'valueToMap')
fig